# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages.
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory.
print(os.getcwd())

# Get your current folder and subfolder event data.
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath.
for root, dirs, files in os.walk(filepath):

# Join the file path and roots with the subdirectories using glob.
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file.
full_data_rows_list = []

# For every filepath in the file path list.
for f in file_path_list:

    # Reading csv file.
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # creating a csv reader object.
        csvreader = csv.reader(csvfile)
        next(csvreader)
        
        # Extracting each data row one by one and append it.
        for line in csvreader:
            # Uncomment the code below if you would like to visualize each of the lines read in the csv files.
            #print(line)
            full_data_rows_list.append(line)

# Uncomment the code below if you would like to get total number of rows.
#print(len(full_data_rows_list))
# Uncomment the code below if you would like to check to see what the list of event data rows will look like.
#print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables.
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the number of rows in your csv file.
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1).
from cassandra.cluster import Cluster

try:
    # Connect to local Apache Cassandra instance.
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session.
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a keyspace called sparkify.
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above.
try:
    session.set_keyspace("sparkify")

except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

---

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
# For the first query, the table will be created containing the songs for a specified session.
# Which has the following column names and their corresponding primary keys:
#   sessionId INT
#   itemInSession INT
#   artist TEXT
#   song TEXT
#   length FLOAT
#   PRIMARY KEY(sessionId, itemInSession)
query_1 = "CREATE TABLE IF NOT EXISTS songs_session "
query_1 = query_1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query_1)

except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below.
file = 'event_datafile_new.csv'

# The file event_datafile_new.csv is opened for reading.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # Skip header.
    next(csvreader)
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable.
        query = "INSERT INTO songs_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement:
        # sessionId = line[8]
        # itemInSession = line[3]
        # artist = line[0]
        # song = line[9]
        # length = line[5]
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Add in the SELECT statement to verify the data was entered into the table songs_session.
# This CQL query only gives me the following:
# The artists with their title and song length that are on the songs_session table when the session ID is 338 and the item in the session is 4.
query = "SELECT artist, song, length FROM songs_session WHERE sessionId = 338 AND itemInSession = 4"

try:
    # The query is executed to obtain the data.
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    # The data corresponding to the artist, the song and its length are displayed.
    print("ARTIST: {}\t SONG: {}\t LENGTH: {}".format(row.artist, row.song, row.length))

ARTIST: Faithless	 SONG: Music Matters (Mark Knight Dub)	 LENGTH: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# For the second query, a table will be created containing the songs per user for a specified session
# Which has the following column names and their corresponding primary keys:
#   userId INT
#   sessionId INT
#   itemInSession INT
#   artist TEXT
#   song TEXT
#   firstName TEXT
#   lastName TEXT
#   PRIMARY KEY((userId, sessionId), itemInSession)
query_2 = "CREATE TABLE IF NOT EXISTS songs_user_session "
query_2 = query_2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query_2)

except Exception as e:
    print(e)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below.
file = 'event_datafile_new.csv'

# The file event_datafile_new.csv is opened for reading.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # Skip header.
    next(csvreader)
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable.
        query = "INSERT INTO songs_user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement:
        # userId = line[10]
        # sessionId = line[8]
        # itemInSession = line[3]
        # artist = line[0]
        # song = line[9]
        # firstName = line[1]
        # lastName = line[4]
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# Add in the SELECT statement to verify the data was entered into the table songs_user_session.
# This CQL query only gives me the following:
# The artists with their song (sorted by the item in the session) and the user (with first and last name) from the songs_user_session table when the user ID is 10 and the session ID is 182.
query = "SELECT artist, song, firstName, lastName, itemInSession FROM songs_user_session WHERE userId = 10 AND sessionId = 182"

try:
    # The query is executed to obtain the data.
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    # The data corresponding to the artist and the song are displayed.
    print("ARTIST: {:20}".format(row.artist), end="")
    print("SONG: {}".format(row.song))

ARTIST: Down To The Bone    SONG: Keep On Keepin' On
ARTIST: Three Drives        SONG: Greece 2000
ARTIST: Sebastien Tellier   SONG: Kilometer
ARTIST: Lonnie Gordon       SONG: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# For the third query, a table will be created containing the users for a specified song.
# Which has the following column names and their corresponding primary keys:
#   song TEXT
#   userId INT
#   firstName TEXT
#   lastName TEXT
#   PRIMARY KEY(song, userId)
query_3 = "CREATE TABLE IF NOT EXISTS users_song "
query_3 = query_3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"

try:
    session.execute(query_3)

except Exception as e:
    print(e)

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below.
file = 'event_datafile_new.csv'

# The file event_datafile_new.csv is opened for reading.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # Skip header.
    next(csvreader)
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable.
        query = "INSERT INTO users_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # song = line[9]
        # userId = line[10]
        # firstName = line[1]
        # lastName = line[4]
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# Add in the SELECT statement to verify the data was entered into the table users_song.
# This CQL query only gives me the following:
# All users (first and last name) from the users_song table when the song is 'All Hands Against His Own'.
query = "SELECT firstName, lastName FROM users_song WHERE song = 'All Hands Against His Own'"

try:
    # The query is executed to obtain the data.
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    # The data corresponding to the user's name are displayed.
    print("FIRST_NAME: {0:10}".format(row.firstname), end="\t")
    print("LAST_NAME: {}".format(row.lastname))

FIRST_NAME: Jacqueline	LAST_NAME: Lynch
FIRST_NAME: Tegan     	LAST_NAME: Levine
FIRST_NAME: Sara      	LAST_NAME: Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop the table songs_session before closing out the sessions
query = "DROP TABLE songs_session"

try:
    rows = session.execute(query)

except Exception as e:
    print(e)

In [18]:
# Drop the table songs_user_session before closing out the sessions
query = "DROP TABLE songs_user_session"

try:
    rows = session.execute(query)

except Exception as e:
    print(e)

In [19]:
# Drop the table users_song before closing out the sessions
query = "DROP TABLE users_song"

try:
    rows = session.execute(query)

except Exception as e:
    print(e)

### Close the session and cluster connection

In [20]:
session.shutdown()
cluster.shutdown()